In [1]:
%load_ext autoreload
%autoreload 2

In [15]:
import requests
import os
from glob import glob
from pprint import pprint

from dotenv import load_dotenv
load_dotenv()

import nest_asyncio
nest_asyncio.apply()

from llama_parse import LlamaParse
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    load_index_from_storage,
    StorageContext
)
from llama_index.core import SummaryIndex, Settings
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.objects import ObjectIndex

from Pycrafter6500.pycrafter6500 import dmd

In [3]:
dlpc9000_docs_dirpath = "../../manuals/dmd"

In [4]:
dmd_docs = {
    os.path.basename(pdf_doc).split(".")[0]: pdf_doc 
    for pdf_doc in glob(os.path.join(dlpc9000_docs_dirpath, "*.pdf"))
}

In [5]:
# set up parser
parser = LlamaParse(
    result_type="markdown"  # "markdown" and "text" are available
)

# use SimpleDirectoryReader to parse manuals
dmd_parsed_docs = {}
for doc_name, doc_path in dmd_docs.items():
    file_extractor = {".pdf": parser, ".html": parser}
    dmd_parsed_docs[doc_name] = SimpleDirectoryReader(input_files=[doc_path], file_extractor=file_extractor).load_data()


Started parsing the file under job_id 86a8633a-c4bb-44f6-b1cf-170091722c71
Started parsing the file under job_id a570114b-8fe5-49ad-bfbd-b5b6cc9bf620


In [6]:
Settings.llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

In [82]:
node_parser = SentenceSplitter()

persist_dir = "./storage/dlpc9000"

if not os.path.exists(persist_dir):
    os.makedirs(persist_dir)

# Build agents dictionary
agents = {}
query_engines = {}

# this is for the baseline
all_nodes = []

# Build agents dictionary
# for doc_name, doc in dmd_parsed_docs.items():

doc_name = 'dlpu018j'
doc = dmd_parsed_docs[doc_name]

print(f"Building agent for document {doc_name}")

nodes = node_parser.get_nodes_from_documents(doc)
all_nodes.extend(nodes)

doc_persist_dir = os.path.join(persist_dir, doc_name)
if not os.path.exists(doc_persist_dir):
    # build vector index
    vector_index = VectorStoreIndex(nodes)
    vector_index.storage_context.persist(
        persist_dir=doc_persist_dir
    )
else:
    vector_index = load_index_from_storage(
        StorageContext.from_defaults(persist_dir=doc_persist_dir),
    )

# build summary index
summary_index = SummaryIndex(nodes)
# define query engines
vector_query_engine = vector_index.as_query_engine(llm=Settings.llm)
summary_query_engine = summary_index.as_query_engine(llm=Settings.llm)

# define tools
query_engine_tools = [
    QueryEngineTool(
        query_engine=vector_query_engine,
        metadata=ToolMetadata(
            name="vector_tool",
            description=,
        ),
    ),
    # QueryEngineTool(
    #     query_engine=summary_query_engine,
    #     metadata=ToolMetadata(
    #         name="summary_tool",
    #         description=summary_query_engine.query("Summarize this document so a tool built upon it knows how to use it.").response,
    #     ),
    # ),
]

# build agent
function_llm = OpenAI(model="gpt-4o")
agent = OpenAIAgent.from_tools(
    query_engine_tools,
    llm=function_llm,
    verbose=True,
    system_prompt=f"""\
            This tools has access to information about communicating with the DLPC900 (aka DLPC9000EVM) board over serial USB. \
            It can go through this document and understand how to format read and write commands to the board. \
            It can answer questions about how to format serial commands for different board commands. \
            It can can also answer questions going in the other direction, given a board command, it will provide the serial command.
            """,
)

agents[doc_name] = agent
query_engines[doc_name] = vector_index.as_query_engine(
    similarity_top_k=2
)

Building agent for document dlpu018j


In [84]:
response = agent.query("What command should I send over USB to set the device to idle?").response
pprint(response)

Added user message to memory: What command should I send over USB to set the device to idle?
=== Calling Function ===
Calling function: vector_tool with args: {"input":"set the device to idle"}
Got output: To set the device to idle, enable Idle Mode. Idle Mode is an operational mode where the entire mirror array is continuously flipped periodically between the on and off states with a 50/50 duty cycle pattern sequence. When Idle Mode is enabled, the LED Enable outputs are disabled to prevent illumination on the DMD. Remember that any pattern sequences must first be stopped before enabling Idle Mode, and it can be enabled in any operating mode except for Video Mode.

('To set the device to idle, you need to enable Idle Mode. Here is the command '
 'to send over USB:\n'
 '\n'
 '```\n'
 '0x1A 0x1B 0x24 0x00 0x01 0x00\n'
 '```\n'
 '\n'
 'This command enables Idle Mode, which flips the entire mirror array '
 'periodically between the on and off states with a 50/50 duty cycle pattern '
 'seq

In [85]:
response = agent.query("What command should I send over USB to turn idle mode off?").response
pprint(response)

Added user message to memory: What command should I send over USB to turn idle mode off?
=== Calling Function ===
Calling function: vector_tool with args: {"input":"turn idle mode off"}
Got output: To turn off Idle Mode, any pattern sequences must first be stopped. Once the pattern sequences are stopped, Idle Mode can be disabled.

('To turn off Idle Mode, you need to follow these steps:\n'
 '\n'
 '1. Stop any pattern sequences that are currently running.\n'
 '2. Disable Idle Mode.\n'
 '\n'
 'Here are the commands you need to send over USB:\n'
 '\n'
 '1. **Stop Pattern Sequence:**\n'
 '   - Command: `0x1A34`\n'
 '   - Data: `0x00`\n'
 '\n'
 '2. **Disable Idle Mode:**\n'
 '   - Command: `0x1A35`\n'
 '   - Data: `0x00`\n'
 '\n'
 'Make sure to send these commands in the specified order to successfully turn '
 'off Idle Mode.')


In [86]:
response = agent.query("What command should I send over USB to turn idle mode on?").response
pprint(response)

Added user message to memory: What command should I send over USB to turn idle mode on?
=== Calling Function ===
Calling function: vector_tool with args: {"input":"turn idle mode on"}
Got output: To turn Idle Mode on, any pattern sequences must first be stopped. Once Idle Mode is enabled, the entire mirror array will continuously flip periodically between the on and off states with a 50/50 duty cycle pattern sequence. Additionally, when Idle Mode is enabled, the LED Enable outputs are disabled to prevent illumination on the DMD.

('To turn Idle Mode on, you need to ensure that any pattern sequences are '
 'stopped first. Once Idle Mode is enabled, the mirror array will continuously '
 'flip between the on and off states with a 50/50 duty cycle pattern sequence, '
 'and the LED Enable outputs will be disabled to prevent illumination on the '
 'DMD.\n'
 '\n'
 'Here is the command to send over USB to turn Idle Mode on:\n'
 '\n'
 '```\n'
 '0x1A 0x34 0x00\n'
 '```\n'
 '\n'
 'This command wi

In [ ]:
[['0x40', '0x0', '0x3', '0x0', '0x1', '0x2', '0x1']]

In [8]:
# define tool for each document agent
all_tools = []
for doc_name, _ in dmd_parsed_docs.items():
    doc_summary = f"This tool is to aid in tasks like the following: {vector_query_engine.query('Finish the last sentence in this message, but don''t include the prompt or the beginning of the sentence. Be as specific as possible but not verbose. This document is useful for accomplishing tasks like the following ').response}"
    doc_tool = QueryEngineTool(
        query_engine=agents[doc_name],
        metadata=ToolMetadata(
            name=f"tool_{doc_name}",
            description=doc_summary,
        ),
    )
    all_tools.append(doc_tool)

In [9]:
obj_index = ObjectIndex.from_objects(
    all_tools,
    index_cls=VectorStoreIndex,
) 

In [77]:
dlp_serial_cmd_interpreter_agent = OpenAIAgent.from_tools(
    tool_retriever=obj_index.as_retriever(similarity_top_k=10),
    system_prompt=f""" \
You are an agent that provides information about commands that are used to control the DLPC900 (aka DLPC9000EVM). You will provide information about formating and specifiying commands over USB. If you are asked about specific commands, the expected format will be a python list of hexadecimal bytes values. ALWAYS BE AS SPECIFIC AS POSSIBLE!! \
You have tools that reference the necessary documentation to interpret the incomding commands. One document is a general user guide and the other is the programmer's guide which contains all of the command definitions. \
Please always use the tools provided to answer a question. Do not rely on prior knowledge. \

""",
    verbose=True,
)

In [16]:
dmd_device = dmd()

/Users/loyalshababo/dev/rtfm/Pycrafter6500/pycrafter6500.py:41: UserWarning: No DLP found. Simulating Device.
  warnings.warning("No DLP found. Simulating Device.")


In [54]:
dmd_device.idle_off()

[['0x40', '0x0', '0x3', '0x0', '0x1', '0x2', '0x0']]


In [55]:
dmd_device.idle_on()

[['0x40', '0x0', '0x3', '0x0', '0x1', '0x2', '0x1']]


In [78]:
dlp_serial_cmd_interpreter_agent.state

AgentState(task_dict={})

In [73]:
dlp_serial_cmd_interpreter_agent.delete_task(task_id = dlp_serial_cmd_interpreter_agent.state.task_dict['2258a15c-dc74-40f2-8cd8-46e057c8ff1f'].task.task_id)

KeyError: '2258a15c-dc74-40f2-8cd8-46e057c8ff1f'

In [75]:
dlp_serial_cmd_interpreter_agent.statedd

AgentState(task_dict={})

In [80]:
response = dlp_serial_cmd_interpreter_agent.query("What command should I send over USB to set the device to idle?")

Added user message to memory: What command should I send over USB to set the device to idle?
=== Calling Function ===
Calling function: tool_dlpu018j with args: {"input":"Get the command to set the device to idle"}
Added user message to memory: Get the command to set the device to idle
=== Calling Function ===
Calling function: summary_tool with args: {"input":"command to set the device to idle"}
Got output: To set the device to idle, use the DMD Idle Mode command.

Got output: To set the device to idle, you should use the **DMD Idle Mode** command.



In [68]:
bytes = "0x1A 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00".split(" ")

In [70]:
len(bytes)

101

In [42]:
response = dlp_serial_cmd_interpreter_agent.query("This is a list of values that would be sent to the DLP900. What command is being given? `['0x40', '0x0', '0x3', '0x0', '0x1', '0x2', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0']`")

Added user message to memory: This is a list of values that would be sent to the DLP900. What command is being given? `['0x40', '0x0', '0x3', '0x0', '0x1', '0x2', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0']`
=== Calling Function ===
Calling function: tool_dlpu018j with args: {"input":"0x40 0x0 0x3 0x0 0x1 0x2 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0"}
Added user message to memory: 0x40 0x0 0x3 0x0 0x1 0x2 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x

In [43]:
pprint(response)

Response(response='The command being given is related to the expanded data in '
                  'the Run-Length Encoding (RLE) Compression Example. This '
                  'sequence represents data that has been decompressed from an '
                  'RLE-compressed format. If you have any further questions or '
                  'need more details about this data, feel free to ask!',
         source_nodes=[NodeWithScore(node=TextNode(id_='e7b072e5-b19f-42f7-ad3e-eac7f2b7d122', embedding=None, metadata={'file_path': '../../manuals/dmd/dlpu018j.pdf', 'file_name': 'dlpu018j.pdf', 'file_type': 'application/pdf', 'file_size': 1728714, 'creation_date': '2024-09-01', 'last_modified_date': '2024-09-01'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelation

In [81]:
for tool in all_tools:
    pprint(tool.metadata)

ToolMetadata(description='This tool is to aid in tasks like the following: '
                         'This document is useful for accomplishing tasks like '
                         'creating and executing batch files to configure '
                         'settings on the DLP LightCrafter 6500 and 9000.',
             name='tool_dlpu018j',
             fn_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>,
             return_direct=False)
ToolMetadata(description='This tool is to aid in tasks like the following: '
                         'This document is useful for accomplishing tasks like '
                         'creating and executing batch files to configure '
                         'settings on the DLP LightCrafter 6500 and 9000.',
             name='tool_dlpu028d',
             fn_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>,
             return_direct=False)
